## IBM 

to_matrix routine in `BasePauli` class.

In [ ]:
import numpy as np
_PARITY = np.array([-1 if bin(i).count("1") % 2 else 1 for i in range(256)], dtype=complex)

def _to_matrix(z, x, phase=0, group_phase=False, sparse=False):
        """Return the matrix from symplectic representation.

        The Pauli is defined as :math:`P = (-i)^{phase + z.x} * Z^z.x^x`
        where ``array = [x, z]``.

        Args:
            z (array): The symplectic representation z vector.
            x (array): The symplectic representation x vector.
            phase (int): Pauli phase.
            group_phase (bool): Optional. If ``True`` use group-phase convention
                                instead of BasePauli ZX-phase convention.
                                (default: ``False``).
            sparse (bool): Optional. Of ``True`` return a sparse CSR matrix,
                           otherwise return a dense Numpy array
                           (default: ``False``).

        Returns:
            array: if ``sparse=False``.
            csr_matrix: if ``sparse=True``.
        """
        num_qubits = z.size

        # Convert to zx_phase
        if group_phase:
            phase += np.sum(x & z)
            phase %= 4

        dim = 2**num_qubits
        twos_array = 1 << np.arange(num_qubits, dtype=np.uint)
        x_indices = np.asarray(x).dot(twos_array)
        z_indices = np.asarray(z).dot(twos_array)

        indptr = np.arange(dim + 1, dtype=np.uint)
        indices = indptr ^ x_indices
        if phase:
            coeff = (-1j) ** phase
        else:
            coeff = 1

        # Compute parities of `z_indices & indptr`, i.e.,
        # np.array([(-1) ** bin(i).count("1") for i in z_indices & indptr])
        vec_u64 = z_indices & indptr
        mat_u8 = np.zeros((vec_u64.size, 8), dtype=np.uint8)
        for i in range(8):
            mat_u8[:, i] = vec_u64 & 255
            vec_u64 >>= 8
            if np.all(vec_u64 == 0):
                break
        parity = _PARITY[np.bitwise_xor.reduce(mat_u8, axis=1)]

        data = coeff * parity
        if sparse:
            # Return sparse matrix
            from scipy.sparse import csr_matrix

            return csr_matrix((data, indices, indptr), shape=(dim, dim), dtype=complex)

        # Build dense matrix using csr format
        mat = np.zeros((dim, dim), dtype=complex)
        mat[range(dim), indices[:dim]] = data[:dim]
        return mat